# Using Tabula Sapiens as a reference for annotating new datasets
This notebook allows you to annotate your data with a number of annotation methods using the Tabula Sapiens dataset as the reference. 

This notebook is also available as a [Google Colab notebook](https://colab.research.google.com/drive/1KEsTbySmXtnOeQo4lJu8OGtPQjHRs1R4#scrollTo=ObY9kyf7exOx)

Integration Methods Provided:
- scVI
- bbKNN
- scanorama

Annotation Methods:
- KNN on integrated spaces
- scANVI
- onClass
- SVM
- RandomForest

**User action is only required in Step 2 and Step 3.**


# Step 1: Setup Environment 
No user input required here.

In [ ]:
import sys
import os
import anndata
import numpy as np
import scanpy as sc
import scvi

# Step 2: Load your data

Load your data into query_adata

In [1]:
# Read in your data with the following command
query_adata = anndata.read("/path/to/query_anndata")


In [ ]:
query_adata

# Step 3: Setting Up Annotation Parameters (User Action Required)

Here is where you set the parameters for the automated annotation.

Arguments:
- **tissue:** Tabula Sapiens tissue to annotate your data with. Available tissues: ["Bladder", "Blood", "Bone_Marrow", "Kidney", "Large_Intestine", "Lung","Lymph_Node", "Pancreas", "Small_Intestine", "Spleen", "Thymus","Trachea", "Vasculature"]
- **save_location:** location to save results to. By default will save to a folder named `annotation_results`. 
- **query_batch_key:** key in `query_adata.obs` for batch correction. Set to None for no batch correction. 
- **methods:** these are the methods to run. By default, will run all methods.
- **training_mode** can be `online` or `offline`. If `offline` will train scVI and scANVI models from scratch. If `online`, will use pretrained models.
- **query_layers_key**: Key in `query_adata.layers` for count data.

Lesser used parameters
- **query_labels_key**: scANVI has the option to use labeled cells in the query dataset during training. To use some prelabeled cells from the query dataset, set `query_labels_key` to the corresponding key in `query_adata.obs`
- **unknown_celltype_label**: If `query_labels_key` is not None, will treat everything not labeled `unknown_celltype_label` as a labeled cell

In [2]:
""" 
tissue options: 
["Bladder", "Blood", "Bone_Marrow", "Kidney", "Large_Intestine", "Lung",
 "Lymph_Node", "Pancreas", "Small_Intestine", "Spleen", "Thymus",
 "Trachea", "Vasculature"]
"""
tissue = 'Lung'
save_folder = 'path/to/save/results/to'
query_batch_key = 'method'
methods = ['bbknn','scvi', 'scanvi', 'svm', 'rf', 'onclass', 'scanorama']
training_mode='offline'
query_layers_key=None


# Lesser used parameters
query_labels_key=None
unknown_celltype_label='unknown'

# Step 4: Downloading Reference Data and Pretrained Models
No more user input required! Just run all the following code blocks.

In [ ]:
# Here we download the necessary data:
if tissue == 'Bladder':
  refdata_url = 'https://ndownloader.figshare.com/files/27388874'
  pretrained_url='https://www.dropbox.com/s/rb89y577l6vs2mm/Bladder.tar.gz?dl=1'
elif tissue == 'Blood':
  refdata_url = 'https://ndownloader.figshare.com/files/27388853'
  pretrained_url = 'https://www.dropbox.com/s/kyh9nv202n0db65/Blood.tar.gz?dl=1'
elif tissue == 'Bone_Marrow':
  refdata_url = 'https://ndownloader.figshare.com/files/27388841'
  pretrained_url = 'https://www.dropbox.com/s/a3r4ddg7o7kua7z/Bone_Marrow.tar.gz?dl=1'
elif tissue == 'Kidney':
  refdata_url = 'https://ndownloader.figshare.com/files/27388838'
  pretrained_url = 'https://www.dropbox.com/s/k41r1a346z0tuip/Kidney.tar.gz?dl=1'
elif tissue == 'Large_Intestine':
  refdata_url = 'https://ndownloader.figshare.com/files/27388835'
  pretrained_url = 'https://www.dropbox.com/s/jwvpk727hd54byd/Large_Intestine.tar.gz?dl=1'
elif tissue == 'Lung':
  refdata_url = 'https://ndownloader.figshare.com/files/27388832'
  pretrained_url = 'https://www.dropbox.com/s/e4al4ia9hm9qtcg/Lung.tar.gz?dl=1'
elif tissue == 'Lymph_Node':
  refdata_url = 'https://ndownloader.figshare.com/files/27388715'
  pretrained_url = 'https://www.dropbox.com/s/mbejy9tcbx9e1yv/Lymph_Node.tar.gz?dl=1'
elif tissue == 'Pancreas':
  refdata_url = 'https://ndownloader.figshare.com/files/27388613'
  pretrained_url = 'https://www.dropbox.com/s/r3klvr22m6kq143/Pancreas.tar.gz?dl=1'
elif tissue == 'Small_Intestine':
  refdata_url = 'https://ndownloader.figshare.com/files/27388559'
  pretrained_url = 'https://www.dropbox.com/s/7eiv2mke70jinzc/Small_Intestine.tar.gz?dl=1'
elif tissue == 'Spleen':
  refdata_url = 'https://ndownloader.figshare.com/files/27388544'
  pretrained_url = 'https://www.dropbox.com/s/6j3iwahsjnb8rb3/Spleen.tar.gz?dl=1'
elif tissue == 'Thymus':
  refdata_url = 'https://ndownloader.figshare.com/files/27388505'
  pretrained_url='https://www.dropbox.com/s/9k0mneu2wvpiudz/Thymus.tar.gz?dl=1'
elif tissue == 'Trachea':
  refdata_url = 'https://ndownloader.figshare.com/files/27388460'
  pretrained_url = 'https://www.dropbox.com/s/57tthfgkl8jtxk6/Trachea.tar.gz?dl=1'
elif tissue == 'Vasculature':
  refdata_url = 'https://ndownloader.figshare.com/files/27388451'
  pretrained_url='https://www.dropbox.com/s/1wt3r871kxjas5o/Vasculature.tar.gz?dl=1'

# Download reference dataset
output_fn = 'TS_{}.h5ad'.format(tissue)
!wget -O $output_fn $refdata_url

# Download pretrained scVI and scANVI models.
output_fn = '{}.tar.gz'.format(tissue)
!wget -O $output_fn $pretrained_url
!tar -xvzf $output_folder

# Download onclass files
!wget -O cl.obo -q https://www.dropbox.com/s/hodp0etapzrd8ak/cl.obo?dl=1 
!wget -O cl.ontology -q https://www.dropbox.com/s/nes0zprzfbwbgj5/cl.ontology?dl=1
!wget -O cl.ontology.nlp.emb https://www.dropbox.com/s/y9x9yt2pi7s0d1n/cl.ontology.nlp.emb?dl=1

# Download annoation code
!wget -O annotation.py -q https://www.dropbox.com/s/id8sallwrunjc5c/annotation.py?dl=1

In [ ]:
# here we setup the query dataset with the reference dataset
import annotation
import importlib
import os
importlib.reload(annotation)
from annotation import process_query

# Following parameters are specific to Tabula Sapiens dataset
ref_labels_key='Annotation'
ref_adata_path = 'TS_{}.h5ad'.format(tissue)
ref_layers_key = 'raw_counts'

pretrained_scanvi_path = os.path.join(tissue, tissue + "_scanvi_model")
pretrained_scvi_path = os.path.join(tissue, tissue + "_scvi_model")

adata = process_query(query_adata,
                      tissue=tissue,
                      save_folder=save_folder,
                      query_batch_key=query_batch_key,
                      query_layers_key=query_layers_key,
                      query_labels_key=query_labels_key,
                      unknown_celltype_label=unknown_celltype_label,
                      pretrained_scvi_path=pretrained_scvi_path,
                      ref_labels_key=ref_labels_key, 
                      ref_layers_key=ref_layers_key,
                      training_mode=training_mode,
                      ref_adata_path=ref_adata_path)

In [ ]:
query_adata


# Step 5: Run Automated Cell Annotation Methods
No user action required. Takes about 30 minutes to run. 

Your results will be saved to the folder you provided as **save_folder**.

There will be the following files:
- `annotated_query.h5ad` containing annotated query cells. The consensus annotations will be in `consensus_prediction`. There will also be a `consensus_percentage` field which is the percentage of methods that had the same prediction. 
- `annotated_query_plus_ref.h5ad` containing your query and the reference cells with predicted annotations. 
- `confusion_matrices.pdf` which contains the confusion matrices between the consensus_predictions and each individual method.
- `csv` files containing the metrics for each confusion matrix. 


In [ ]:
import annotation
importlib.reload(annotation)
from annotation import annotate_data

annotate_data(adata,
              methods, 
              save_folder,
              pretrained_scvi_path=pretrained_scvi_path,
              pretrained_scanvi_path=pretrained_scanvi_path)

# Step 6 Generate Statistics and Figures
No user action required.

## Agreements
First we define some variables from the results files to make the code cleaner 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

results_file = os.path.join(save_folder,'annotated_query_plus_ref.h5ad')
results = anndata.read(results_file)
is_query = results.obs._dataset == "query"
methods = [x for x in results.obs.columns if x.endswith("_pred")]
labels = results.obs.consensus_prediction.astype(str)
labels[~is_query] = results[~is_query].obs._labels_annotation.astype(str)
celltypes = np.unique(labels)
latent_methods = results.obsm.keys()

### Distribution of consensus percentage
The more the algorithms agree with each other, the better the annotation has worked


In [ ]:
agreement_counts = pd.DataFrame(
    np.unique(results[is_query].obs["consensus_percentage"], return_counts=True)
).T

agreement_counts.columns = ["Percent Agreement", "Count"]
agreement_counts.plot.bar(
    x="Percent Agreement", y="Count", legend=False, figsize=(4, 3)
)
plt.ylabel("Frequency")
plt.xlabel("Percent of Algorithms Agreeing with Majority Vote")
figpath = os.path.join(save_folder, "Concensus_Percentage_barplot.pdf")
plt.savefig(figpath, bbox_inches="tight")

### Per cell type agreement
Some cell types can be better predicted than others, and we can highlight the celltypes that are poorly predicted by looking at the per celltype agreement. The cell types are separated by the concensus predictions.

In [ ]:
mean_agreement = [
    np.mean(results[is_query & (labels == x)].obs["consensus_percentage"].astype(float))
    for x in celltypes
]
mean_agreement = pd.DataFrame([mean_agreement], index=["agreement"]).T
mean_agreement.index = celltypes

mean_agreement = mean_agreement.sort_values("agreement", ascending=True)
mean_agreement.plot.bar(y="agreement", figsize=(15, 2), legend=False)
plt.ylabel("Mean Agreement")
plt.xticks(rotation=290, ha="left")
figpath = os.path.join(save_folder, "percelltype_agreement_barplot.pdf")
plt.savefig(figpath, bbox_inches="tight")

### Cell type proportion plot

In [ ]:
prop = pd.DataFrame(index=celltypes, columns=["ref", "query"])
for x in celltypes:
    prop.loc[x, "query"] = np.sum(labels[is_query] == x)
    prop.loc[x, "ref"] = np.sum(labels[~is_query] == x)


In [ ]:
prop.loc[mean_agreement.index].plot(kind='bar', figsize=(len(celltypes)*0.5,4),logy=True)
plt.legend(bbox_to_anchor=(1, 0.9))
plt.ylabel('log Celltype Abundance')
plt.tight_layout()
figpath = os.path.join(save_folder, 'celltype_prop_barplot.pdf')
plt.savefig(figpath, bbox_inches="tight")
plt.show()
plt.close()
